In [ ]:
from disqco.circuits.cp_fraction import cp_fraction
from disqco import QuantumCircuitHyperGraph
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume  
from disqco import QuantumNetwork
from disqco.graphs.quantum_network import linear_coupling, grid_coupling, random_coupling, all_to_all
from disqco.parti import FiducciaMattheyses
from disqco.graphs.coarsening.coarsener import HypergraphCoarsener
import numpy as np

# Circuit setup
num_qubits = 64
circuit = cp_fraction(num_qubits=num_qubits, depth=num_qubits, fraction=0.5)
circuit = transpile(circuit, basis_gates=['cp', 'u'])

# Partition setup
num_partitions = 16
qpu_sizes = [int(num_qubits/num_partitions)+1] * num_partitions


graph = QuantumCircuitHyperGraph(circuit)

print(f"Circuit: {num_qubits} qubits, depth {circuit.depth()}")
print(f"Partitions: {num_partitions} QPUs, sizes: {qpu_sizes}")
print(f"Hypergraph: {len(graph.nodes)} nodes, {len(graph.hyperedges)} edges")

 

In [ ]:
# Create different network topologies
topologies = {
    'All-to-All': all_to_all(num_partitions),
    'Linear': linear_coupling(num_partitions),
    'Grid': grid_coupling(num_partitions),
    'Random': random_coupling(num_partitions, p=0.4)
}

networks = {}
for name, coupling in topologies.items():
    networks[name] = QuantumNetwork(qpu_sizes, coupling)
    print(f"\n{name} Network:")
    print(f"  Nodes: {list(networks[name].qpu_graph.nodes())}")
    print(f"  Edges: {list(networks[name].qpu_graph.edges())}")

In [ ]:
# Visualize each network topology
import matplotlib.pyplot as plt



for i, (name, network) in enumerate(networks.items()):

    network.draw()


plt.tight_layout()
plt.show()


In [ ]:
# Run multilevel partitioning for each network topology
coarsener = HypergraphCoarsener()
results = {}

for name, network in networks.items():
    print(f"\n{'='*50}")
    print(f"Running multilevel partitioning for {name} network")
    print(f"{'='*50}")
    
    # Initialize partitioner
    partitioner = FiducciaMattheyses(
        circuit=circuit,
        hypergraph=graph,
        network=network
    )
    
    # Run multilevel partitioning
    result = partitioner.multilevel_partition(
        coarsener=coarsener.coarsen_recursive_batches_mapped,
    )
    
    results[name] = result
    print(f"Final cost for {name}: {result['best_cost']}")

In [ ]:
# Visualize cost evolution for each topology
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Cost evolution per level
for name, result in results.items():
    cost_list = result['cost_list']
    ax1.plot(range(len(cost_list)), cost_list, 'o-', label=name, linewidth=2, markersize=6)

ax1.set_xlabel('Coarsening Level')
ax1.set_ylabel('Cost')
ax1.set_title('Cost Evolution Across Coarsening Levels')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Final cost comparison
topologies_names = list(results.keys())
final_costs = [results[name]['best_cost'] for name in topologies_names]
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold']

bars = ax2.bar(topologies_names, final_costs, color=colors, alpha=0.7, edgecolor='black')
ax2.set_ylabel('Final Cost')
ax2.set_title('Final Cost Comparison')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, cost in zip(bars, final_costs):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{cost}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()